In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("workding dir:", workding_dir)

workding dir: /home/inflaton/code/projects/courses/novel-translation


In [ ]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: /home/inflaton/code/projects/courses/novel-translation/.env


True

In [ ]:
import os

model_name = os.getenv("MODEL_NAME")
token = os.getenv("HF_TOKEN") or None
load_in_4bit = os.getenv("LOAD_IN_4BIT") == "true"
local_model = os.getenv("LOCAL_MODEL")
hub_model = os.getenv("HUB_MODEL")
num_train_epochs = int(os.getenv("NUM_TRAIN_EPOCHS") or 0)
data_path = os.getenv("DATA_PATH")

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)

model_name, load_in_4bit, local_model, hub_model, max_seq_length, num_train_epochs, dtype, data_path

('unsloth/Qwen2-0.5B-Instruct',
 True,
 'models/Qwen2-0.5B-Instruct-MAC-',
 'Qwen2-0.5B-Instruct-MAC-',
 2048,
 10,
 None,
 'datasets/mac/mac.tsv')

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [ ]:
%%time

from llm_translation.translation_engine import *

model, tokenizer = load_model(model_name, load_in_4bit)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Qwen2 patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 4080 Laptop GPU. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 10.6 s, sys: 2.07 s, total: 12.6 s
Wall time: 51.9 s


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
%%time

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2024.5 patched 24 layers with 0 QKV layers, 24 O layers and 24 MLP layers.


CPU times: user 9.31 s, sys: 0 ns, total: 9.31 s
Wall time: 2.12 s


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
import os
from llm_translation.translation_engine import *

datasets = load_translation_dataset(data_path, tokenizer)

loading train/test data files
DatasetDict({
    train: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 4528
    })
    test: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 1133
    })
})


In [ ]:
datasets["train"][0], datasets["test"][0]

({'chinese': '全仗着狐仙搭救。',
  'english': 'Because I was protected by a fox fairy.',
  'text': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n全仗着狐仙搭救。<|im_end|>\n<|im_start|>assistant\nBecause I was protected by a fox fairy.<|im_end|>',
  'prompt': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n全仗着狐仙搭救。<|im_end|>\n<|im_start|>assistant\n'},
 {'chinese': '老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。',
  'english': 'Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.',
  'text': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>

In [ ]:
datasets["train"][1000], datasets["test"][1000]

({'chinese': '周瑞家的道：“太太说：‘他们原不是一家子； 当年他们的祖和太老爷在一处做官，因连了宗的。',
  'english': "'She said they don't really belong to the family but were adopted into the clan years ago when your grandfather and theirs were working in the same office.",
  'text': "<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n周瑞家的道：“太太说：‘他们原不是一家子； 当年他们的祖和太老爷在一处做官，因连了宗的。<|im_end|>\n<|im_start|>assistant\n'She said they don't really belong to the family but were adopted into the clan years ago when your grandfather and theirs were working in the same office.<|im_end|>",
  'prompt': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n周瑞家的道：“太太说：‘他们原不是一家子； 当年他们的祖和太老爷在一处做官，因连了宗的。<|im_end|>\n<|im_start|>assistant\n'},
 {'chinese': '“听到了吗？',
  'english': "'Did you hear that?'",
  'text': "<|im_start|>system\nYou are an expert in translating Chinese i

In [ ]:
%%time

prompt1 = datasets["test"]["prompt"][1000]
print(prompt1)
print("--" * 20)
test_model(model, tokenizer, prompt1)

<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
“听到了吗？<|im_end|>
<|im_start|>assistant

----------------------------------------
<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
“听到了吗？<|im_end|>
<|im_start|>assistant
Did you hear it?<|im_end|>
CPU times: user 1.62 s, sys: 160 ms, total: 1.78 s
Wall time: 1.8 s


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from llm_translation.transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=datasets["train"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=num_train_epochs,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
import torch

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4080 Laptop GPU. Max memory = 11.994 GB.
1.199 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,528 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 5,660
 "-____-"     Number of trainable parameters = 8,798,208


Step,Training Loss
100,2.137700
200,2.001500
300,1.938200
400,1.935400
500,1.899800
600,1.819500
700,1.689600
800,1.737300
900,1.665900
1000,1.664600


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3779.1598 seconds used for training.
62.99 minutes used for training.
Peak reserved memory = 1.855 GB.
Peak reserved memory for training = 0.656 GB.
Peak reserved memory % of max memory = 15.466 %.
Peak reserved memory for training % of max memory = 5.469 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
%%time

prompt1 = datasets["test"]["prompt"][0]
print(prompt1)
print("--" * 20)
test_model(model, tokenizer, prompt1)

<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant

----------------------------------------
<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant
Old Geng lifted his gun and squinted over his shoulder, because the pistol started firing.<|im_end|>
CPU times: user 1.71 s, sys: 161 ms, total: 1.87 s
Wall time: 1.86 s


In [ ]:
print(datasets["test"]["english"][0])

Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.


In [ ]:
%%time

predictions = eval_model(model, tokenizer, datasets["test"])

100%|██████████| 1133/1133 [26:28<00:00,  1.40s/it]

CPU times: user 23min 55s, sys: 2min 33s, total: 26min 28s
Wall time: 26min 28s


In [ ]:
calc_metrics(datasets["test"]["english"], predictions, debug=True)

{'accuracy': 0.00088261253309797,
 'correct_ids': [147],
 'bleu_scores': {'bleu': 0.06508609399238363,
  'precisions': [0.3407579117113485,
   0.09377291935878182,
   0.03598822203642444,
   0.01652015762352228],
  'brevity_penalty': 0.9858565320713017,
  'length_ratio': 0.9859556144418682,
  'translation_length': 29766,
  'reference_length': 30190},
 'rouge_scores': {'rouge1': 0.32340459562777546,
  'rouge2': 0.11259712507132531,
  'rougeL': 0.2671219091010598,
  'rougeLsum': 0.2670685844265569}}

In [ ]:
from llm_translation.translation_engine import save_results

save_results(model_name + "(finetuned)", "results/mac-results.tsv", datasets["test"], predictions, debug=True)

                                             chinese  \
0  老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞...   

                                             english  \
0  Old Geng picked up his shotgun, squinted, and ...   

              unsloth/Qwen2-0.5B-Instruct(finetuned)  
0  Old Geng lifted his rifle and narrowed his eye...  
